# Template for subclassing `param.Parametrized` class 
Useful for creating a visuzliation gui to quickly explore different parameter spaces and its effect on your model

## Load standard libraries


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import pandas as pd
    
from pathlib import Path #we'll import Path object with `.ls` method added later
from pprint import pprint as pp

import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
)

## Here is the template!

In [ ]:
CITIES = ['LA', 'BOSTON', 'PARIS', 'JEONJU', 'DC', 'SF']
class MyParamedClass(pm.Parameterized):
    
    # Declare parameters (which will turned into instance attributes via. pm.Parametrized.__init__ method)
    age = pm.Integer(10, bounds=(1,100))
    city = pm.Selector(objects=CITIES, default='LA')
    logbox = pm.String('LOGBOX')
    
    # Explicitly define initialization method
    ## This will be called to instantiate new instances for this class
    def __init__(self, **params):
        """
        The signature of this init method matched pm.Parametrized class's init method:
            pm.Parametrized.__init__(self, **params)
        This allows to specify parameter values at instaniation using 'key'=value: 
            eg: me = MyParamedClass(age=11, city='JEONJU')
            
        See pm.Parametrized?? for details
        """
        super().__init__(**params)
        self.country = 'USA'
        
    # Annotate a method with its dependencies that does not need automatic update when the dependents' values change
    @param.depends('age', watch=False) # watch=False is default
    def get_stage(self):
        stage = 'old' if self.age > 50 else 'young'
        return stage
    
    # Annotate method that needs be updated automatically as its dependents' parameter values change
    @param.depends('city', watch=True)
    def update_country(self):
        if self.city in ['LA', 'BOSTON', 'DC', 'SF']:
            self.country = 'USA'
        elif self.city == 'PARIS':
            self.country = 'FRANCE'
        else:
            self.country = 'KOREA'
        print(f"Country is updated: {self.country}")
        
        # Just for demonstrating that this method is actually called when `city` parameter changes
        self.logbox = self.country
        print('Logbox: ', self.logbox)
        
                
    
    
        
    

In [ ]:
myex = MyParamedClass()

In [ ]:
pn.panel(myex.param)#.servable()

In [ ]:
pn.panel(myex.param['logbox'])

Enjoy:)